# DATA101 Guided Activity (75 to 90 minutes)

## Data + Task Abstraction to Visualization Design



**Instructor:** Marc Reyes (marc.reyes@dlsu.edu.ph)



### What you will practice

- Writing a task spec (Action + Target + Constraints + Output)

- Writing a data spec (dataset type + variable types + transforms)

- Implementing the transforms in Python

- Producing 2 small charts that directly support your tasks



### What you will submit

By the end, you should have:

1) A completed **task spec** (in this notebook)

2) A completed **data spec** (in this notebook)

3) Two charts (time comparison + ranked change)

4) A short justification (4 to 6 sentences) linking abstractions to design



This notebook uses a **synthetic dataset** (generated below) so we can focus on abstraction and method rather than data access.

## Scenario (Domain Question)



A department head asks:



> "Are students struggling more this term compared to last term?"



Your job is to translate that domain question into:

- a **task abstraction** (what actions are needed)

- a **data abstraction** (what structure and variables you need)

- then a visualization design you can defend.



### Time budget (recommended)

- 10 min: write tasks + outputs

- 20 min: data inventory + typing

- 25 min: transforms + derived measures

- 25 min: charts + quick interpretation

- 10 min: justification + cleanup



If you finish early, try the stretch goal at the end.

In [ ]:
# Setup

# If something fails to import on your JupyterHub, message your instructor with the error text.



import math



try:

    import numpy as np

except ImportError as e:

    raise ImportError("This activity requires numpy.") from e



try:

    import pandas as pd

except ImportError as e:

    raise ImportError("This activity requires pandas.") from e



try:

    import matplotlib.pyplot as plt

except ImportError as e:

    raise ImportError("This activity requires matplotlib.") from e



pd.set_option("display.max_columns", 50)

pd.set_option("display.width", 120)



try:

    plt.style.use("seaborn-v0_8-whitegrid")

except Exception:

    # If seaborn styles are not available, keep matplotlib defaults.

    pass



plt.rcParams.update({

    "figure.dpi": 120,

    "axes.titlesize": 12,

    "axes.labelsize": 11,

    "font.size": 11,

})

In [ ]:
# Create a synthetic dataset: one row per (term, program, section, week).

# This matches a common "table + time + category" structure from the lecture.



def make_student_week_data(seed: int = 101) -> pd.DataFrame:

    rng = np.random.default_rng(seed)



    terms = ["2024-T3", "2025-T1"]  # baseline, current

    weeks = np.arange(1, 13)

    programs = ["CS", "DS", "IS", "IT"]

    sections_per_program = 2



    # Program effects (baseline differences)

    program_effect = {

        "CS": 0.02,

        "DS": 0.00,

        "IS": 0.01,

        "IT": -0.01,

    }



    rows = []

    for term in terms:

        for program in programs:

            for section_idx in range(sections_per_program):

                section_id = f"{program}-{chr(ord('A') + section_idx)}"

                course_code = "DATA101"



                base_n = int(rng.integers(25, 55))

                for week in weeks:

                    # Enrollment fluctuates a bit week to week

                    n_students = int(np.clip(round(base_n + rng.normal(0, 3)), 15, 80))



                    # Baseline trend: slight decline after week 6 (fatigue)

                    fatigue = -0.01 * max(0, week - 6)



                    # Term effect: current term is slightly lower overall

                    term_effect = -0.02 if term == "2025-T1" else 0.0



                    # A stronger shock for DS and IT after week 7 in the current term

                    shock = 0.0

                    if term == "2025-T1" and program in {"DS", "IT"} and week >= 7:

                        shock = -0.07 - 0.01 * (week - 7)



                    # Construct a pass_rate with noise and clip to valid range

                    pass_rate_true = 0.84 + program_effect[program] + fatigue + term_effect + shock

                    pass_rate_true = float(np.clip(pass_rate_true + rng.normal(0, 0.03), 0.35, 0.98))



                    n_pass = int(rng.binomial(n_students, pass_rate_true))

                    avg_score = float(np.clip(55 + 50 * pass_rate_true + rng.normal(0, 5), 0, 100))



                    rows.append({

                        "term": term,

                        "week": int(week),

                        "program": program,

                        "course_code": course_code,

                        "section_id": section_id,

                        "n_students": n_students,

                        "n_pass": n_pass,

                        "avg_score": avg_score,

                    })



    return pd.DataFrame(rows)





df_raw = make_student_week_data(seed=101)

df_raw.head()

# Part 1 - Task Abstraction (write before charting)



Edit the cell below and write your answers in plain English.



Remember the structure:

- **Action**: compare, rank, detect, locate, summarize, filter

- **Target**: items, groups, time ranges, attributes

- **Constraints**: baseline, subgroup, top-k, time window

- **Output**: verifiable result (flagged weeks, ranked list, etc.)



Tip: Do not write chart names here. Write what the user needs to do.

## Your Task Spec (edit this cell)



**Domain question (1 sentence):**

- ...



**Task 1 (Monitoring):**

- Action: ...

- Target: ...

- Constraints (baseline + scope): ...

- Output (verifiable): ...



**Task 2 (Discovery):**

- Action: ...

- Target: ...

- Constraints (baseline + scope): ...

- Output (verifiable): ...



**What decision should your output support?**

- Example: Identify which programs need intervention and which weeks to investigate.

# Part 2 - Data Abstraction (inventory + types + transforms)



First, treat this like a professional data inventory.



Questions to answer:

- What is one row (unit of analysis)?

- What are the main variables?

- Which variables are categorical, quantitative, or temporal?

- What transforms are required before charting (derive rates, aggregate, compare vs baseline)?

In [ ]:
# Data inventory

df_raw.shape, df_raw.columns.tolist()

In [ ]:
# Look at a few rows (sanity check)

df_raw.sample(8, random_state=7)

In [ ]:
# Current dtypes

df_raw.dtypes

## Task: Fix types (5 to 10 minutes)



In the lecture: **type mistakes create misleading charts**.



TODO:

- Convert these to categorical: `term`, `program`, `course_code`, `section_id`

- Ensure these are integers: `week`, `n_students`, `n_pass`

- Keep `avg_score` as a float (quantitative)



Then run the checks below.

In [ ]:
df = df_raw.copy()

# TODO: convert to category: term, program, course_code, section_id
# Hint: df[col] = df[col].astype("category")
# YOUR CODE HERE

# TODO: ensure integers: week, n_students, n_pass
# Hint: df[col] = df[col].astype(int)
# YOUR CODE HERE

df.dtypes


In [ ]:
# Checks (should run without errors)

assert df["term"].dtype.name == "category"

assert df["program"].dtype.name == "category"

assert df["course_code"].dtype.name == "category"

assert df["section_id"].dtype.name == "category"

assert df["week"].dtype.kind in {"i", "u"}

assert df["n_students"].dtype.kind in {"i", "u"}

assert df["n_pass"].dtype.kind in {"i", "u"}



df.head(3)

## Your Data Spec (edit this cell)



**Primary dataset type:**

- ...



**Unit of analysis (one row means):**

- ...



**Variables + attribute types:**

- `term` = ...

- `week` = ...

- `program` = ...

- `section_id` = ...

- `n_students` = ...

- `n_pass` = ...

- `avg_score` = ...



**Required transforms (before charting):**

- ...



Tip: include at least one **derived measure** and at least one **aggregation** step.

# Part 3 - Transforms (derive + aggregate + compare vs baseline)



In this dataset, raw counts (`n_pass`, `n_students`) are not directly comparable unless you convert them to a **rate**.



You will:

1) derive `pass_rate`

2) aggregate to **program-week** (because many questions are about groups)

3) compute change vs baseline term



This matches the workflow from the slides: question -> tasks -> data needs -> transforms -> design.

In [ ]:
# 1) Derived measure
# TODO: create pass_rate = n_pass / n_students (float)
# YOUR CODE HERE

# Basic sanity checks
assert "pass_rate" in df.columns
assert df["pass_rate"].between(0, 1).all()
df[["term", "week", "program", "section_id", "n_students", "n_pass", "pass_rate"]].head(5)


In [ ]:
# 2) Aggregate to program-week for comparison
# TODO: compute a table with one row per (term, program, week).
# Required columns: term, program, week, n_students, n_pass, avg_score, pass_rate
# - Sum n_students
# - Sum n_pass
# - avg_score as mean
# - pass_rate from summed counts (n_pass / n_students)
# Hint: df.groupby(["term", "program", "week"], as_index=False).agg(...)

# YOUR CODE HERE

program_week.head(8)


In [ ]:
# 3) Compute change vs baseline (delta)
# Goal: a table with columns: program, week, pass_rate_baseline, pass_rate_current, delta_pass_rate

baseline_term = "2024-T3"
current_term = "2025-T1"

# TODO: create `pivot` from `program_week`.
# Hint: program_week.pivot_table(index=["program", "week"], columns="term", values="pass_rate")
# Then rename the baseline/current columns and compute delta_pass_rate.

# YOUR CODE HERE

pivot.head(8)


In [ ]:
# Task output 1: detect when each program "starts" declining
# Define a simple rule: the first week where delta <= -0.08

threshold = -0.08

# TODO: create `first_drop` with columns: program, week, delta_pass_rate
# Hint: sort by week, create a boolean flag, then take the first flagged row per program.

# YOUR CODE HERE

first_drop


In [ ]:
# Task output 2: rank programs by average change in weeks 7-12

# TODO: filter weeks 7-12, then compute mean delta_pass_rate per program, then sort ascending.
# Output columns: program, avg_delta_pass_rate

# YOUR CODE HERE

ranked


# Part 4 - Design and Charts



Now that you have tasks and data transforms, you can justify encodings:

- `week` is temporal (ordered), so it belongs on the x-axis.

- `pass_rate` is quantitative, so position on a common y-scale is a safe comparison channel.

- `program` is categorical, so you can use facets (small multiples) or color grouping.



You will produce two charts:

1) A time comparison chart (current vs baseline)

2) A ranked change chart (which programs declined the most)



Keep it simple. The goal is defendability, not decoration.

In [ ]:
# Chart 1: time comparison per program (small multiples)
# Run this after you finish Part 3 (program_week must exist).

baseline_term = "2024-T3"
current_term = "2025-T1"

term_colors = {baseline_term: "#6b7280", current_term: "#2563eb"}  # gray, blue
programs = sorted(program_week["program"].unique())

ncols = 2
nrows = math.ceil(len(programs) / ncols)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(11, 6), sharex=True, sharey=True)
axes = np.array(axes).reshape(-1)

for ax, program in zip(axes, programs):
    sub = program_week[program_week["program"] == program].sort_values(["term", "week"])
    for term in [baseline_term, current_term]:
        s = sub[sub["term"] == term]
        ax.plot(
            s["week"],
            s["pass_rate"],
            color=term_colors[term],
            linewidth=2,
            marker="o",
            markersize=3.5,
            label=term,
        )
    ax.set_title(f"Program: {program}")
    ax.set_xlabel("Week")
    ax.set_ylabel("Pass rate")
    ax.set_ylim(0.35, 1.0)
    ax.axvline(7, color="#111827", linewidth=1, alpha=0.25)
    ax.text(7.1, 0.36, "wk7", fontsize=9, alpha=0.6)

# Hide any unused axes
for ax in axes[len(programs):]:
    ax.axis("off")

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper center", ncol=2, frameon=False)
fig.suptitle("Pass rate over time: current vs baseline", y=1.02)
fig.tight_layout()
plt.show()


In [ ]:
# Chart 2: ranked average change (weeks 7-12)



ranked_plot = ranked.sort_values("avg_delta_pass_rate")

y = np.arange(len(ranked_plot))



fig, ax = plt.subplots(figsize=(8, 4.5))

ax.hlines(y, xmin=0, xmax=ranked_plot["avg_delta_pass_rate"], color="#93c5fd", linewidth=5, alpha=0.9)

ax.scatter(ranked_plot["avg_delta_pass_rate"], y, color="#1d4ed8", s=45, zorder=3)



ax.axvline(0, color="#111827", linewidth=1, alpha=0.35)

ax.set_yticks(y)

ax.set_yticklabels(ranked_plot["program"])

ax.set_xlabel("Average delta pass rate (current - baseline), weeks 7-12")

ax.set_title("Which programs declined the most? (negative is worse)")



for i, v in enumerate(ranked_plot["avg_delta_pass_rate"]):

    ax.text(v + (0.005 if v >= 0 else -0.005), i, f"{v:+.3f}", va="center", ha="left" if v >= 0 else "right", fontsize=9, alpha=0.8)



fig.tight_layout()

plt.show()

# Part 5 - Interpretation + Justification (write-up)



Edit the cell below and answer briefly.



Your write-up must link: **tasks -> data -> transforms -> design**.



Keep it specific and verifiable.

## Your Write-Up (edit this cell)



1) **What did you find?**

- Which program(s) show the largest decline?

- Around what week does the decline begin (based on your rule)?



2) **Why these charts? (4 to 6 sentences)**

- Mention at least one variable type -> encoding match (example: quantitative -> position).

- Mention at least one transform that was necessary (example: rate, baseline delta).

- Mention how the design supports each task output.



3) **One improvement you would make**

- Example: add uncertainty, show denominators, add annotations for known events, or test another threshold.

# Stretch Goal (optional, 10 to 15 minutes)



Pick one:

- Add a third chart that shows **distribution**, not only averages (example: box plot of section pass_rate by program for weeks 7-12).

- Change the task from program-level to section-level and see if conclusions change.

- Replace the simple threshold rule with a rolling average rule (still explainable).